# For this demo, we will simulate differential equations from the lecture

## First, import some library
The key function here is [odeint](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html)

In [ ]:
from scipy.integrate import odeint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Model 1: Fixed transcription rate and a fixed degradation rate
$$\frac{d[RNA]}{dt} = k_\text{transcription} - k_\text{degradation}[RNA]$$

Define a function that compute the differential function

Note that our function does not depend on **time**

In [ ]:
def simple_transcription(rna, time = np.arange(0, 20, 1), k_trans = 1.0, k_deg = 0.5):
    return k_trans - k_deg * rna

### To perform a simulation, we have to define the following:
1. Initial [RNA]
2. Timesteps: default = range from 0 to 20, with step size = 1
3. $k_\text{transcriptio}$: default = 1.0
4. $k_\text{degradation}$: default = 0.5

### Task 1: Try varying the initial [RNA] and constants

In [ ]:
initial_rna = 0.01 ## initial RNA concentration

k_trans = 1 ## transcription rate
k_deg = 0.5 ## degradation rate

times = np.arange(0, 20, 1)
simulated = odeint(simple_transcription, initial_rna, times, args = (k_trans, k_deg))

plt.plot(times, simulated)
plt.xlabel('time'); plt.ylabel('[RNA]')
plt.show()

### One key setting for a simulation is the resolution of the time scale
Generally, the timesteps must correspond to the units of $k_\text{transcriptio}$ and $k_\text{degradation}$ 

In this demo, **we need to adjust the timesteps** to see the detailed behavior of the system

In [ ]:
## This system will converge to an equilibrium very quickly
initial_rna = 0
k_trans = 100
k_deg = 15

## Rough timestep = 1.0
times_rough = np.arange(0, 3.01, 1)
simulated_rough = odeint(simple_transcription, initial_rna, times_rough, args = (k_trans, k_deg))

## Fine timestep = 0.01
times_fine = np.arange(0, 3, 0.01)
simulated_fine = odeint(simple_transcription, initial_rna, times_fine, args = (k_trans, k_deg))

## Compare the resulting simulations
plt.figure(figsize = (10, 4))

plt.subplot(1, 2, 1)
plt.plot(times_rough, simulated_rough)
plt.xlabel('time'); plt.ylabel('[RNA]'); plt.title('Rough time step')

plt.subplot(1, 2, 2)
plt.plot(times_fine, simulated_fine)
plt.xlabel('time'); plt.ylabel('[RNA]'); plt.title('Fine time step')

plt.tight_layout()
plt.show()

## Adding time-dependent activation
$$\frac{d[RNA]}{dt} = k_\text{transcription} - k_\text{degradation}[RNA]\text{, for } t \leq 5$$
$$\frac{d[RNA]}{dt} = - k_\text{degradation}[RNA]\text{, for } t > 5$$

Add an **if-else** statement to the function

In [ ]:
def time_transcription(rna, time, k_trans, k_deg):
    if time < 5:
        return k_trans - k_deg * rna
    else:
        return - k_deg * rna

### Expression level peaks at t = 5, followed by an exponential decay

In [ ]:
initial_rna = 0
k_trans = 2
k_deg = 0.5

times = range(0, 30)
simulated = odeint(time_transcription, initial_rna, times, args = (k_trans, k_deg))

plt.plot(times, simulated)
plt.xlabel('time'); plt.ylabel('[RNA]')
plt.show()

## Model 2: Negative auto-regulation
$$\frac{d[RNA]}{dt} = \frac{k_\text{transcription}}{1 + (k_\text{binding}[RNA])^n} - k_\text{degradation}[RNA]$$

In [ ]:
def neg_auto_transcription(rna, time, k_trans, k_deg, k_nar, n = 2):
    return k_trans / (1 + (k_nar * rna) ** n) - k_deg * rna

### Task 2: Try changing initial [RNA] to investigate the behavior of the system

In [ ]:
initial_rna = 1
k_trans = 2
k_deg = 0.5
k_nar = 1
n = 3

times = np.arange(0, 20, 0.1)
simulated = odeint(neg_auto_transcription, initial_rna, times, args = (k_trans, k_deg, k_nar, n))

plt.plot(times, simulated)
plt.xlabel('time'); plt.ylabel('[RNA]')
plt.show()

## Compare behavior of a negative auto-regulation system from multiple initial conditions
At low [RNA], the expression level rises until the equilibrium

At high [RNA], the negative auto-regulation mechanism represses the expression

### Task 3: Can you adjust the parameter(s) to make the system more complex

In [ ]:
times = np.arange(0, 8, 0.1)

k_trans = 2
k_deg = 0.5
k_nar = 1
n = 3

for initial_rna in np.arange(0, 4, 0.1):
    simulated = odeint(neg_auto_transcription, initial_rna, times, args = (k_trans, k_deg, k_nar, n))
    
    if simulated[0] < simulated[-1]:
        plt.plot(times, simulated, c = 'tab:blue')
    else:
        plt.plot(times, simulated, c = 'tab:red')

plt.xlabel('time'); plt.ylabel('[RNA]')
plt.show()

## Model 3: Positive auto-regulation
$$\frac{d[RNA]}{dt} = \frac{k_\text{transcription}(k_\text{binding}[RNA])^n}{1 + (k_\text{binding}[RNA])^n} - k_\text{degradation}[RNA]$$

In [ ]:
def pos_auto_transcription(rna, time, k_trans, k_deg, k_nar, n):
    return k_trans * (k_nar * rna) ** n / (1 + (k_nar * rna) ** n) - k_deg * rna

### Task 4: Try changing initial [RNA] to investigate the behavior of the system

In [ ]:
initial_rna = 0
times = np.arange(0, 30, 0.01)

k_trans = 2
k_deg = 0.5
k_nar = 1
n = 2

simulated = odeint(pos_auto_transcription, initial_rna, times, args = (k_trans, k_deg, k_nar, n))

plt.plot(times, simulated)
plt.xlabel('time'); plt.ylabel('[RNA]')
plt.show()

### Bistability via positive auto-regulation

In [ ]:
k_trans = 2
k_deg = 0.5
k_nar = 1
n = 10

times = np.arange(0, 30, 0.01)

for initial_rna in np.arange(0, 6, 0.1):
    simulated = odeint(pos_auto_transcription, initial_rna, times, args = (k_trans, k_deg, k_nar, n))
    
    if simulated[-1] < 0.5:
        plt.plot(times, simulated, c = 'tab:orange')
    elif simulated[0] < simulated[-1]:
        plt.plot(times, simulated, c = 'tab:blue')
    else:
        plt.plot(times, simulated, c = 'tab:red')

plt.xlabel('time'); plt.ylabel('[RNA]')
plt.show()

## Model 4: Gene toggle switch
Two genes repressing each other

Assume the same transcription rate, degradation rate, and dimerization

$$\frac{d[LacI]}{dt} = \frac{k_\text{transcription}}{1 + [GFP]^2} - k_\text{degradation}[LacI]$$
$$\frac{d[GFP]}{dt} = \frac{k_\text{transcription}}{1 + [LacI]^2} - k_\text{degradation}[GFP]$$

In [ ]:
def toggle_switch(rna, time, k_trans, k_deg, n = 2):
    LacI = rna[0]
    GFP = rna[1]
    
    dLacI_dt = k_trans / (1 + GFP ** n) - k_deg * LacI
    dGFP_dt = k_trans / (1 + LacI ** n) - k_deg * GFP
    
    return [dLacI_dt, dGFP_dt]

### Task 5: Vary the two initial [RNA] to study the dynamics

In [ ]:
initial_rna = [5, 4]
k_trans = 2
k_deg = 0.5

times = np.arange(0, 30, 0.01)
simulated = odeint(toggle_switch, initial_rna, times, args = (k_trans, k_deg))

plt.plot(times, simulated[:, 0], label = 'LacI')
plt.plot(times, simulated[:, 1], label = 'GFP')
plt.xlabel('time'); plt.ylabel('[RNA]'); plt.legend()
plt.show()

## Intervene the system by neutralizing LacI molecules
Start at time = 10

In [ ]:
def toggle_switch_intervene(rna, time, k_trans, k_deg, n):
    LacI = rna[0]
    GFP = rna[1]
    
    dLacI_dt = k_trans / (1 + GFP ** n) - k_deg * LacI
    
    if time < 10:
        dGFP_dt = k_trans / (1 + LacI ** n) - k_deg * GFP
    else:
        dGFP_dt = k_trans - k_deg * GFP
    
    return [dLacI_dt, dGFP_dt]

In [ ]:
initial_rna = [2, 1]
times = np.arange(0, 30, 0.01)

k_trans = 2
k_deg = 0.5
n = 2

simulated = odeint(toggle_switch_intervene, initial_rna, times, args = (k_trans, k_deg, n))

plt.plot(times, simulated[:, 0], label = 'LacI')
plt.plot(times, simulated[:, 1], label = 'GFP')
plt.xlabel('time step'); plt.ylabel('[RNA]'); plt.legend()
plt.show()

## Model 5: Two genes with only linear effect
$$\frac{d[x_1]}{dt} = k_{11}[x_1] + k_{12}[x_2]$$
$$\frac{d[x_2]}{dt} = k_{21}[x_1] + k_{22}[x_2]$$

In [ ]:
def two_loci_linear(rna, time, k11, k12, k21, k22):
    dx1_dt = k11 * rna[0] + k12 * rna[1]
    dx2_dt = k21 * rna[0] + k22 * rna[1]
    
    return [dx1_dt, dx2_dt]

## Function for visualizing the dynamics over time
def view_simulation(simulated, times):
    plt.figure(figsize = (8, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(times, simulated[:, 0], label = 'x1')
    plt.plot(times, simulated[:, 1], label = 'x2')
    plt.xlabel('time'); plt.ylabel('[X]'); plt.legend()
    plt.title('expression over time')

    plt.subplot(1, 2, 2)
    plt.scatter(simulated[:, 0], simulated[:, 1], c = times, s = 0.1)
    plt.xlabel('[X1]'); plt.ylabel('[X2]')
    plt.title('dynamics over time (yellow = late time steps)')

    plt.tight_layout()
    plt.show()

### Task 6: Vary k11, k12, k21, and k22 to achieve different system behaviors

In [ ]:
initial_x = [2, 1]
times = np.arange(0, 50, 0.01)

k11 = -0.4
k12 = 0.6
k21 = -0.2
k22 = 0.3

simulated = odeint(two_loci_linear, initial_x, times, args = (k11, k12, k21, k22))
view_simulation(simulated, times)

### Can you tell how the behavior of the system is determined by the values k11, k12, k21, and k22?